Prepare our HTTP API client

In [10]:
module HttpClient =
    open System 
    open System.Net.Http 
    open System.IO

    type RequestMethod = 
        | Get
        | Post 

    let client = new System.Net.Http.HttpClient()

    let buildQueryFromMap (parameters: Map<string, string>) =
        use content = new FormUrlEncodedContent(parameters)
        let stream = content.ReadAsStream()
        let reader = new StreamReader(stream)
        reader.ReadToEnd()

    let handleResponseAsync req = 
        async {
            let! response = client.SendAsync(req) |> Async.AwaitTask
            response.EnsureSuccessStatusCode() |> ignore 

            return!
                response.Content.ReadAsStringAsync()
                |> Async.AwaitTask
        }

    let getRequestAsync (endpoint: string) (someHeaders: Map<string, string> option) (someQueryParameters: Map<string, string> option) = 

        let req = new HttpRequestMessage()
        req.Method <- HttpMethod.Get

        match someHeaders with 
        | Some headers -> 
            headers 
            |> Map.toSeq 
            |> Seq.iter (fun (k, v) -> 
                req.Headers.Add(k, v)
            )
        | None -> 
            ()

        match someQueryParameters with 
        | Some parameters -> 
            req.RequestUri <- 
                Uri(endpoint + "?" + (buildQueryFromMap parameters))
        | None -> 
            req.RequestUri <- 
                Uri(endpoint)



        handleResponseAsync req


    let postRequestAsync (endpoint: string) (someHeaders: Map<string, string> option) (someQueryParameters: Map<string, string> option) (someBody: Map<string, string> option) = 

        let req = new HttpRequestMessage()
        req.Method <- HttpMethod.Get

        match someHeaders with 
        | Some headers -> 
            headers 
            |> Map.toSeq 
            |> Seq.iter (fun (k, v) -> 
                req.Headers.Add(k, v)
            )
        | None -> 
            ()

        match someQueryParameters with 
        | Some parameters -> 
            req.RequestUri <- 
                Uri(endpoint + "?" + (buildQueryFromMap parameters))
        | None -> 
            req.RequestUri <- 
                Uri(endpoint)

        match someBody with 
        | Some body -> 
            req.Content <- new FormUrlEncodedContent(body)
        | None -> 
            ()

        handleResponseAsync req

Define our Travel module which find coordiate from place

In [11]:
module Travel = 
    let locate place = 
        let queryOptions:  Map<string, string> = 
            Map.empty
                .Add("q", $"{place}")
                .Add("format", "json")
                .Add("addressdetails", "1")

        async {
            return! HttpClient.getRequestAsync "https://nominatim.openstreetmap.org/search" None (Some queryOptions)
        }
        

        
        

In [12]:
Travel.locate "丹东市, 振兴区" |> Async.RunSynchronously

Error: System.Net.Http.HttpRequestException: Response status code does not indicate success: 403 (Forbidden).
   at System.Net.Http.HttpResponseMessage.EnsureSuccessStatusCode()
   at FSI_0020.HttpClient.handleResponseAsync@21-4.Invoke(HttpResponseMessage response)
   at Microsoft.FSharp.Control.AsyncPrimitives.CallThenInvokeNoHijackCheck[a,b](AsyncActivation`1 ctxt, b result1, FSharpFunc`2 userCode) in D:\a\_work\1\s\src\FSharp.Core\async.fs:line 528
   at Microsoft.FSharp.Control.Trampoline.Execute(FSharpFunc`2 firstAction) in D:\a\_work\1\s\src\FSharp.Core\async.fs:line 148
--- End of stack trace from previous location ---
   at Microsoft.FSharp.Control.AsyncResult`1.Commit() in D:\a\_work\1\s\src\FSharp.Core\async.fs:line 454
   at Microsoft.FSharp.Control.AsyncPrimitives.RunImmediate[a](CancellationToken cancellationToken, FSharpAsync`1 computation) in D:\a\_work\1\s\src\FSharp.Core\async.fs:line 1160
   at Microsoft.FSharp.Control.AsyncPrimitives.RunSynchronously[T](Cance